# CI
At first, you'll want to write your tests locally, and test them against as many local browsers as possible. However, to really test out your features, you'll want to:

- run them against as many **real** browsers on other operating systems as possible
- have easy access to human- and machine-readable test results and build assets
- integration with development tools like GitHub

Enter Continuous Integration (CI). 

## Providers: Cloud

### Multi-Provider
Historically, Jupyter projects have used a mix of free-as-in-beer-for-open source hosted services:
- [Appveyor](https://www.appveyor.com) for Windows
- [Circle-CI](https://circleci.com) for Linux
- [TravisCI](https://travis-ci.org) for Linux and MacOS

Each brings their own syntax, features, and constraints to building and maintaining robust CI workflows.

> `JupyterLibrary` started on Travis-CI, but as soon as we wanted to support more platforms and browsers...

### Azure Pipelines
At the risk of putting all your eggs in one (proprietary) basket, [Azure Pipelines](https://azure.microsoft.com/en-us/services/devops/pipelines/) provides a single-file approach to automating all of your tests against reasonably modern versions of browsers. 

> `JupyterLibrary` was formerly built on Azure, and looking through [pipeline][] and various [jobs and steps][] shows some evolving approaches...

[pipeline]: https://github.com/robots-from-jupyter/robotframework-jupyterlibrary/blob/v0.2.0/azure-pipelines.yml
[jobs and steps]: https://github.com/robots-from-jupyter/robotframework-jupyterlibrary/tree/v0.2.0/ci

### Github Actions
At the risk of putting all your eggs in one (proprietary) basket, if your code is on Github, [Github Actions](https://github.com/features/actions) offers the tightest integration, requiring no aditional accounts.

> `JupyterLibrary` is itself built on Github Actions, and looking at the [workflows][] offers some of the best patterns we have found.

[workflows]: https://github.com/robots-from-jupyter/robotframework-jupyterlibrary/blob/main/.github/workflows/ci.yml

## Providers: On-Premises

### Jenkins
If you are working on in-house projects, and/or have the ability to support it, [Jenkins](https://jenkins.io) is the gold standard for self-hosted continuous integration. It has almost limitless configurability, and commercial support is available.

- [`warnings-ng`](https://plugins.jenkins.io/warnings-ng/) can consume many outputs of `robotframework`

## Approach: Environment management

Acceptance tests need benefit from tightly-controlled, but flexibly-defined environments. 

- this repo uses (and recommends) `conda-lock` and `mamba` to manage multiple environments
- simpler cases, such as pure-python projects, can use [`tox`](https://github.com/tox-dev/tox)

## Approach: It's Just Scripts
No matter how shiny or magical your continuous integration tools appear, the long-term well-being of your repo depends on techniques that are: 
- simple
- cross-platform
- as close to real browsers as possible
- easily reproducible outside of CI

Practically, since this is Jupyter, this boils down to putting as much as possible into platform-independent python (and, when neccessary, nodejs) code. 

> `JupyterLibrary` uses [doit][] to manage a relatively complex lifecycle across multiple environments with minimal CLI churn.
>
> - `doit` has very few runtime dependencies, and works well with caching, etc.
> 
> Environment variables are used for feature flags
>
>  - aside from some inevitable path issues, environment variables are easy to migrate onto another CI provider
>
> A small collection of development [scripts][], not shipped as part of the distribution, provide some custom behaviors around particularly complex tasks.
> 
> - sometimes `doit` is too heavy of a hammer for delicate work

[scripts]: https://github.com/robots-from-jupyter/robotframework-jupyterlibrary/tree/main/scripts
[doit]: https://github.com/robots-from-jupyter/robotframework-jupyterlibrary/blob/main/dodo.py

## Approach: Single Test Script

Having a single command that runs _all_ unit, integration, and acceptance tests is a useful property of a project.

- `make` (or the more pythonic [`doit`](https://github.com/pydoit/doit), used in this repo) make this most robust
  - usually, all _unit_ tests need to be re-run when any functional source, e.g. `*.ts` and `*.py`
  - acceptance tests often need to be run when almost _anything_ changes, including `.css`, build configuration files, etc.
- wrap `robot` execution in another tool
  - for example, [jupyter-server-proxy] launches `robot` from within `pytest`
  - use [`tox`](https://github.com/tox-dev/tox) for pure-python test management 
  
[jupyter-server-proxy]: https://github.com/jupyterhub/jupyter-server-proxy/blob/v3.2.2/tests/acceptance/test_acceptance.py

## Approach: Log Centralization

After a full test run, it can be useful to combine many test results into a single, navigable page

- in CI, download all the test result archives and put them together
  - [`rebot`][rebot] can combine multiple runs, including retries, into a single HTML report
- embed different kinds of results
  - [`pytest-html`][pytest-html] can embed generated reports
    - when embedding `robot` reports with screenshots, use [`Set Screenshot Directory   EMBED`][embed] to make this easier
    - other files like logs can also be embedded
- create a single log aggregation HTML page
    - [jupyterlab-deck] generates and publishes a notebook/slideshow containing all of its logs
        - this is served as a [JupyterLite] site, so the underlying (semi-)machine-readable is also available to 

[rebot]: https://robotframework.org/robotframework/latest/RobotFrameworkUserGuide.html#using-rebot
[pytest-html]: https://github.com/pytest-dev/pytest-html
[embed]: https://robotframework.org/SeleniumLibrary/SeleniumLibrary.html#Set%20Screenshot%20Directory
[jupyterlab-deck]: https://deathbeds.github.io/jupyterlab-deck
[jupyterlite]: https://github.com/jupyterlite/jupyterlite

## Approach: Caching
Most of the CI providers offer nuanced approaches to caching files. Things to try caching (it doesn't always help):
- packages/metadata for your package manager, e.g. `conda`, `pip`, `yarn`
- built web assets

## Approach: Pay technical debt forward

A heavy CI pipeline can become necessary to manage many competing concerns. Each non-trivial, browser-based robot test can easily cost tens of seconds. Some approaches:
- use an up-front dry-run `robot` test
  - this can help catch whitespace errors in robot syntax
  - this usually costs $\frac{\sim1}{100}$ the time of running the full test
- run tests in subsets, in parallel, and in random order with [`pabot`](https://github.com/mkorpela/pabot)
  - requires avoiding shared resources, e.g. network ports, databases, logfiles
    - if neccessary, declare explicit dependencies with e.g. [`DependencyLibrary`](https://pypi.org/project/robotframework-dependencylibrary) or [`pabot`'s `#DEPENDS`](https://github.com/mkorpela/pabot#controlling-execution-order-and-level-of-parallelism)

## Approach: Get More Value

While the pass/fail results of a test are useful in their own right, acceptance tests can provide useful artifacts for other project goals.

- gather additional coverage insrumentation
  - [x] **client**:
    - [x] [jupyterlab-deck][deck-cov] uses `istanbul` and `nyc` to collect browser code coverage
  - [x] **kernel** and **widgets**:
    - [x] this repo gathers kernel coverage from JupyterLab-based tests iof its custom `%%robot` [IPython magic][ipython-magic]
    - [x] [ipyforcegraph](https://github.com/jupyrdf/ipyforcegraph/pull/89) tests custom [Jupyter widgets][widgets]
  - [ ] _**serverextension**: TODO_
  - [ ] _**`.robot` suites**: TODO_
- use generated screenshots
  - [ ] _**reporting**: TODO_
  - [ ] _**accessibility**: TODO_ 
  - [ ] _**documentation**: TODO_
  - [ ] _**PDF generation**: TODO_
    - [ ] revisit when supported by [`geckodriver`](https://github.com/mozilla/geckodriver/issues/2001)  

[deck-cov]: https://deathbeds.github.io/jupyterlab-deck/files/nyc/index.html
[deck-screens]: https://deathbeds.github.io/jupyterlab-deck/lab/index.html?path=README.ipynb
[ipython-magic]: https://ipython.readthedocs.io/en/stable/interactive/magics.html
[widgets]: https://github.com/jupyter-widgets/ipywidgets